In [ ]:
"""
Purpose: To Run the neuron preprocessing


"""

In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

# configuring the virtual module

In [2]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
#time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2020-11-26 01:26:40,987 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-26 01:26:40,988 - settings - Setting database.user to celiib
INFO - 2020-11-26 01:26:40,990 - settings - Setting database.password to newceliipass
INFO - 2020-11-26 01:26:40,994 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2020-11-26 01:26:40,995 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-26 01:26:41,007 - connection - Connect

Sleeping 65 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons
Connecting celiib@at-database.ad.bcm.edu:3306


# Defining Our Table

In [3]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [4]:
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 8442


In [5]:
#so that it will have the adapter defined
from datajoint_utils import *

In [6]:
#minnie.Decomposition.drop()
#schema.external['decomposition'].delete(delete_external_files=True)

In [17]:
import numpy as np
import time
decimation_version = 0
decimation_ratio = 0.25

@schema
class Decomposition(dj.Computed):
    definition="""
    -> minnie.Decimation.proj(decimation_version='version')
    ---
    decomposition: <decomposition>
    n_vertices           : int unsigned                 # number of vertices
    n_faces              : int unsigned                 # number of faces
    n_error_limbs: int #the number of limbs that are touching multiple somas or 1 soma in multiple places
    n_same_soma_multi_touching_limbs: int # number of limbs that touch the same soma multiple times
    n_multi_soma_touching_limbs: int # number of limbs that touch multiple somas
    n_somas: int #number of soma meshes detected
    n_limbs: int
    n_branches: int
    max_limb_n_branches=NULL:int
    
    skeletal_length=NULL: double
    max_limb_skeletal_length=NULL:double
    median_branch_length=NULL:double #gives information on average skeletal length to next branch point
    
    
    width_median=NULL: double #median width from mesh center without spines removed
    width_no_spine_median=NULL: double #median width from mesh center with spines removed
    width_90_perc=NULL: double # 90th percentile for width without spines removed
    width_no_spine_90_perc=NULL: double  # 90th percentile for width with spines removed
    
    
    n_spines: bigint

    spine_density=NULL: double # n_spines/ skeletal_length
    spines_per_branch=NULL: double
    
    skeletal_length_eligible=NULL: double # the skeletal length for all branches searched for spines
    n_spine_eligible_branches=NULL: int # the number of branches that were checked for spines because passed width threshold
    
    spine_density_eligible=NULL:double # n_spines/skeletal_length_eligible
    spines_per_branch_eligible=NULL:double # n_spines/n_spine_eligible_branches
    
    total_spine_volume=NULL: double # the sum of all spine volume
    spine_volume_median=NULL: double # median of the spine volume for those spines with able to calculate volume
    spine_volume_density=NULL: double #total_spine_volume/skeletal_length
    spine_volume_density_eligible=NULL: double #total_spine_volume/skeletal_length_eligible
    spine_volume_per_branch_eligible=NULL: double #total_spine_volume/n_spine_eligible_branches
    
    run_time=NULL : double                   # the amount of time to run (seconds)

    
    """

#     key_source =  ((minnie.Decimation).proj(decimation_version='version') & 
#                             "decimation_version=" + str(decimation_version) &
#                        f"decimation_ratio={decimation_ratio}" &  (minnie.BaylorSegmentCentroid() & "multiplicity>0" & "segment_id=864691136309663834").proj())
    #key_source = (minnie.Decimation() & "n_faces>500000").proj(decimation_version='version') & (minnie.BaylorSegmentCentroid() & "multiplicity=1").proj()
    
    import test_neurons as tn
    neurons_to_process = np.concatenate([getattr(tn,f) for f in dir(tn) if "one_cell" in f])
    key_source = (minnie.Decimation()).proj(decimation_version='version') & (minnie.BaylorSegmentCentroid() & "multiplicity=1").proj() & [dict(segment_id=k) for k in neurons_to_process]
    

    def make(self,key):
        """
        Pseudocode for process:

        1) Get the segment id from the key
        2) Get the decimated mesh
        3) Get the somas info
        4) Run the preprocessing
        5) Calculate all starter stats
        6) Save the file in a certain location
        7) Pass stats and file location to insert
        """
        whole_pass_time = time.time()
        #1) Get the segment id from the key
        segment_id = key["segment_id"]
        description = str(key['decimation_version']) + "_25"
        print(f"\n\n----- Working on {segment_id}-------")
        global_start = time.time()
        
        #2) Get the decimated mesh
        current_neuron_mesh = du.fetch_segment_id_mesh(segment_id,minnie=minnie)

        #3) Get the somas info *************************** Need to change this when actually run *******************
        somas = du.get_soma_mesh_list(segment_id,minnie=minnie) 
        print(f"somas = {somas}")
        #4) Run the preprocessing


        total_neuron_process_time = time.time()

        print(f"\n--- Beginning preprocessing of {segment_id}---")
        recovered_neuron = neuron.Neuron(
        mesh = current_neuron_mesh,
        somas = somas,
        segment_id=segment_id,
        description=description,
        suppress_preprocessing_print=False,
        suppress_output=False,
        calculate_spines=True,
        widths_to_calculate=["no_spine_median_mesh_center"]

                )

        print(f"\n\n\n---- Total preprocessing time = {time.time() - total_neuron_process_time}")


        #5) Don't have to do any of the processing anymore because will do in the neuron object
        stats_dict = recovered_neuron.neuron_stats()



        #6) Save the file in a certain location
        save_time = time.time()
        ret_file_path = recovered_neuron.save_compressed_neuron(output_folder=str(du.get_decomposition_path()),
                                          return_file_path=True,
                                         export_mesh=False,
                                         suppress_output=True)

        ret_file_path_str = str(ret_file_path.absolute()) + ".pbz2"
        print(f"Save time = {time.time() - save_time}")



        #7) Pass stats and file location to insert
        new_key = dict(key,
                       decomposition=ret_file_path_str,
                       n_vertices=len(current_neuron_mesh.vertices),
                       n_faces=len(current_neuron_mesh.faces),
                       run_time=np.round(time.time() - whole_pass_time,4)
                      )
        new_key.update(stats_dict)

        self.insert1(new_key, allow_direct_insert=True, skip_duplicates=True)

        print(f"\n\n ------ Total time for {segment_id} = {time.time() - global_start} ------")
    

# Running The Populate

In [18]:
#(minnie.schema.jobs & "table_name='__decomposition'").delete()
#((schema.jobs & "table_name = '__decomposition'") & "timestamp>'2020-11-16 00:26:00'").delete()

In [20]:
import time
import random
import compartment_utils as cu
cu = reload(cu)
import preprocessing_vp2 as pre
pre = reload(pre)

start_time = time.time()
#time.sleep(random.randint(0, 900))
print('Populate Started')
Decomposition.populate(reserve_jobs=True, suppress_errors=False, order='random')
print('Populate Done')

print(f"Total time for Decomposition populate = {time.time() - start_time}")

Populate Started


----- Working on 864691134884740858-------
somas = [[<trimesh.Trimesh(vertices.shape=(16738, 3), faces.shape=(33297, 3))>], array([376.9315]), array([0.773])]

--- Beginning preprocessing of 864691134884740858---
--- 0) Having to preprocess the Neuron becuase no preprocessed data
Please wait this could take a while.....
Skipping the hole filling
use_meshafterparty = True
Using pre-computed somas: soma_mesh_list = [<trimesh.Trimesh(vertices.shape=(16738, 3), faces.shape=(33297, 3))>]
Soma List = [<trimesh.Trimesh(vertices.shape=(16738, 3), faces.shape=(33297, 3))>]
soma_mesh_list_centers = [array([621316.50090214, 628142.56355598, 784118.88921018])]
xvfb-run -n 8442 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Datajoint/Neuron_Processing/temp/neuron_12420.off -o /notebooks/Platinum_Datajoint/Neuron_Processing/temp/neuron_12420_fill_holes.off -s /notebooks/Platinum_Datajoint/Neuron_Processing/temp/fill_holes_282130.mls

---- meshlab output -----
Q

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:16<00:00, 16.05s/it]

Time for 1st pass MP skeletonization: 17.99751925468445
branches_touching_root = [9]
length of Graph = 43801
Working on path [1852. 1868. 1881. 1895. 1924. 1954. 1975. 1986.]
path_degrees = [3, 2, 2, 2, 2, 2, 2, 3]
Working on path [3519. 3530. 3541. 3546.]
path_degrees = [3, 2, 2, 3]
Working on path [11757. 11761. 11764. 11769. 11771.]
path_degrees = [3, 2, 2, 2, 3]
Working on path [13314. 13321. 13322. 13328. 13329.]
path_degrees = [3, 2, 2, 2, 3]
Working on path [20853. 20863. 20870.]
path_degrees = [3, 2, 3]
Working on path [21262. 21278. 21294.]
path_degrees = [3, 2, 3]
Working on path [23672. 23692. 23730. 23750. 23760.]
path_degrees = [3, 2, 2, 2, 3]
Working on path [25235. 25258. 25269. 25281. 25294. 25305. 25317. 25327.]
path_degrees = [3, 2, 2, 2, 2, 2, 2, 3]
Working on path [29347. 29349. 29357. 29364. 29366.]
path_degrees = [3, 2, 2, 2, 3]
Working on path [34221. 34227.]
path_degrees = [3, 3]
Working on path [42752. 42760. 42764. 42765.]
path_degrees = [3, 2, 2, 3]
max(kept_


AFTER face_lookup_resolved_test


...


There was only one mesh found from the spine process and mesh split, returning empty array



There was only one mesh found from the spine process and mesh split, returning empty array



There was only one mesh found from the spine process and mesh split, returning empty array



Working on limb L0 branch 3
No spines and using precomputed width
Working on limb L0 branch 4
Working on limb L0 branch 10
Working on limb L0 branch 191
Working on limb L0 branch 192
Working on limb L0 branch 8
Working on limb L0 branch 201
Working on limb L0 branch 189
No spines and using precomputed width
Working on limb L0 branch 190
Working on limb L0 branch 2
No spines and using precomputed width
Working on limb L0 branch 197
No spines and using precomputed width
Working on limb L0 branch 1
Working on limb L0 branch 5
Working on limb L0 branch 194
Working on limb L0 branch 195
No spines and using precomputed width
Working on limb L0 branch 9
Working on limb L0 branch 23
Working on limb L0 branch 193
No spines and using precomputed width
Working on limb L0 branch 196
Working on limb L0 branch 7
Working on limb L0 branch 200
No spines and using precomputed width
Working on limb L0 branch 22
Working on limb L0 branch 119
No spines and using precomputed width
Working on limb L0 branc

Working on limb L0 branch 48
No spines and using precomputed width
Working on limb L0 branch 71
No spines and using precomputed width
Working on limb L0 branch 94
No spines and using precomputed width
Working on limb L0 branch 123
No spines and using precomputed width
Working on limb L0 branch 131
No spines and using precomputed width
Working on limb L0 branch 180
No spines and using precomputed width
Working on limb L0 branch 35
No spines and using precomputed width
Working on limb L0 branch 156
No spines and using precomputed width
Working on limb L0 branch 153
No spines and using precomputed width
Working on limb L0 branch 159
No spines and using precomputed width
Working on limb L0 branch 47
No spines and using precomputed width
Working on limb L0 branch 86
No spines and using precomputed width
Working on limb L0 branch 70
No spines and using precomputed width
Working on limb L0 branch 96
No spines and using precomputed width
Working on limb L0 branch 93
No spines and using precomp